In [1]:
import pandas as pd
import numpy as np

In [4]:
data=pd.read_json("restaurant_data.json")
data.head()

,token,pos,head,deprel,aspects
0,"[But, the, staff, was, so, horrible, to, us, .]","[CC, DT, NN, VBD, RB, JJ, TO, PRP, .]","[6, 3, 6, 6, 6, 0, 8, 6, 6]","[cc, det, nsubj, cop, advmod, root, case, nmod...","[{'term': ['staff'], 'from': 2, 'to': 3, 'pola..."
1,"[To, be, completely, fair, ,, the, only, redee...","[TO, VB, RB, JJ, ,, DT, JJ, JJ, NN, VBD, DT, N...","[4, 4, 4, 12, 12, 9, 9, 9, 12, 12, 12, 0, 12, ...","[mark, cop, advmod, advcl, punct, det, amod, a...","[{'term': ['food'], 'from': 11, 'to': 12, 'pol..."
2,"[The, food, is, uniformly, exceptional, ,, wit...","[DT, NN, VBZ, RB, JJ, ,, IN, DT, RB, JJ, NN, W...","[2, 5, 5, 5, 0, 5, 11, 11, 10, 11, 5, 15, 15, ...","[det, nsubj, cop, advmod, root, punct, case, d...","[{'term': ['food'], 'from': 1, 'to': 2, 'polar..."
3,"[Not, only, was, the, food, outstanding, ,, bu...","[RB, RB, VBD, DT, NN, JJ, ,, CC, DT, JJ, ``, N...","[2, 3, 0, 5, 3, 5, 3, 3, 10, 12, 10, 15, 12, 1...","[neg, cc, root, det, nsubj, amod, punct, cc, d...","[{'term': ['food'], 'from': 4, 'to': 5, 'polar..."
4,"[Our, agreed, favorite, is, the, orrechiete, w...","[PRP$, VBN, NN, VBZ, DT, NN, IN, NN, CC, NN, -...","[3, 3, 6, 6, 6, 0, 8, 6, 8, 8, 16, 16, 14, 16,...","[nmod, amod, nsubj, cop, det, root, case, nmod...","[{'term': ['orrechiete', 'with', 'sausage', 'a..."


In [5]:
X=pd.DataFrame(columns=['tweets'])
X['tweets']=data['token']
X['tweets']=X['tweets'].apply(lambda x:' '.join(x))
X

,tweets
0,But the staff was so horrible to us .
1,"To be completely fair , the only redeeming fac..."
2,"The food is uniformly exceptional , with a ver..."
3,"Not only was the food outstanding , but the li..."
4,Our agreed favorite is the orrechiete with sau...
...,...
1975,"The service was typical short-order , dinner t..."
1976,"We shared a bottle of sake , an order of edama..."
1977,I ca n't believe people complain about no chee...
1978,"From the appetizers we ate , the dim sum and o..."


In [8]:
y=pd.DataFrame(columns=['polarity'])
for i in data['aspects']:
    print(i)
    for j in i:
        y=y._append({'polarity':j['polarity']},ignore_index=True)

[{'term': ['staff'], 'from': 2, 'to': 3, 'polarity': 'negative'}]
[{'term': ['food'], 'from': 11, 'to': 12, 'polarity': 'positive'}]
[{'term': ['food'], 'from': 1, 'to': 2, 'polarity': 'positive'}, {'term': ['kitchen'], 'from': 10, 'to': 11, 'polarity': 'positive'}, {'term': ['menu'], 'from': 27, 'to': 28, 'polarity': 'neutral'}]
[{'term': ['food'], 'from': 4, 'to': 5, 'polarity': 'positive'}, {'term': ["'", 'perks'], 'from': 10, 'to': 12, 'polarity': 'positive'}]
[{'term': ['orrechiete', 'with', 'sausage', 'and', 'chicken'], 'from': 5, 'to': 10, 'polarity': 'positive'}, {'term': ['waiters'], 'from': 13, 'to': 14, 'polarity': 'positive'}, {'term': ['meats'], 'from': 29, 'to': 30, 'polarity': 'neutral'}, {'term': ['dish'], 'from': 20, 'to': 21, 'polarity': 'neutral'}]
[{'term': ['Bagels'], 'from': 1, 'to': 2, 'polarity': 'positive'}]
[{'term': ['food'], 'from': 2, 'to': 3, 'polarity': 'positive'}]
[{'term': ['songs'], 'from': 13, 'to': 14, 'polarity': 'neutral'}]
[{'term': ['toast'], 'f

In [7]:
y['polarity']

0       negative
1       positive
2       positive
3       positive
4        neutral
          ...   
3603     neutral
3604     neutral
3605     neutral
3606     neutral
3607     neutral
Name: polarity, Length: 3608, dtype: object

In [7]:
from sklearn.preprocessing import OneHotEncoder
ohe=OneHotEncoder(drop='first')
y_new=ohe.fit_transform(y[['polarity']]).toarray()
y_new

array([[0., 0.],
       [0., 1.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]])

In [8]:
print(X.shape,y.shape)


(6051, 1) (6051, 1)


In [24]:
from keras_preprocessing.text import Tokenizer
from keras import Sequential
from keras.layers import Dense,SimpleRNN,Embedding
from keras.utils import pad_sequences 

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X=X['tweets']
y=y_new

In [12]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42)

In [13]:
print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(4538,) (4538, 2)
(1513,) (1513, 2)


# Tokenizing the text

In [14]:
tokenizer=Tokenizer(num_words=999,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',lower=True,split=" ")

In [15]:
tokenizer.fit_on_texts(X)

In [16]:
X_train_tok=tokenizer.texts_to_sequences(X_train)
X_test_tok=tokenizer.texts_to_sequences(X_test)

In [17]:
X_test_tok[0]

[302, 264, 82, 776, 24, 185, 280, 469, 3, 1, 111, 2, 527, 5, 90, 564]

In [18]:
X_train.head()

332       katy perry wants to crash the royal wedding , .
833     when you compare lindsay lohan pre and post co...
2582    Wave An Amazing list of all the google product...
5709    industry games - guitar hero brand respected l...
2611    the wii sucks , its fun for a couple of weeks ...
Name: tweets, dtype: object

In [19]:
X_train_tok[0]

[274, 278, 391, 3, 2, 852]

In [20]:
X_train_pad=pad_sequences(X_train_tok)
X_test_pad=pad_sequences(X_test_tok)

In [21]:
X_train_pad[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0, 274, 278, 391,   3,   2, 852])

In [22]:
X_test_pad[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0, 302, 264,  82, 776,  24, 185, 280, 469,   3,   1, 111,
         2, 527,   5,  90, 564])

# Model Training

In [34]:
model=Sequential()
model.add(Embedding(2000,2))
model.add(SimpleRNN(64,return_sequences=False))
model.add(Dense(2,activation="softmax"))
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 2)           4000      
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, 64)                4288      
                                                                 
 dense_3 (Dense)             (None, 2)                 130       
                                                                 
Total params: 8,418
Trainable params: 8,418
Non-trainable params: 0
_________________________________________________________________


In [35]:
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [36]:
model.fit(X_train_pad,y_train,epochs=10,validation_data=[X_test_pad,y_test])

Epoch 1/10
142/142 [==============================] - 2s 10ms/step - loss: 0.4797 - accuracy: 0.7488 - val_loss: 0.4824 - val_accuracy: 0.7568
Epoch 2/10
142/142 [==============================] - 1s 8ms/step - loss: 0.4790 - accuracy: 0.7490 - val_loss: 0.4782 - val_accuracy: 0.7568
Epoch 3/10
142/142 [==============================] - 1s 8ms/step - loss: 0.4795 - accuracy: 0.7490 - val_loss: 0.4770 - val_accuracy: 0.7568
Epoch 4/10
142/142 [==============================] - 1s 8ms/step - loss: 0.4797 - accuracy: 0.7490 - val_loss: 0.4758 - val_accuracy: 0.7568
Epoch 5/10
142/142 [==============================] - 1s 8ms/step - loss: 0.4799 - accuracy: 0.7490 - val_loss: 0.4826 - val_accuracy: 0.7568
Epoch 6/10
142/142 [==============================] - 1s 9ms/step - loss: 0.4809 - accuracy: 0.7490 - val_loss: 0.4749 - val_accuracy: 0.7568
Epoch 7/10
142/142 [==============================] - 1s 8ms/step - loss: 0.4790 - accuracy: 0.7490 - val_loss: 0.4757 - val_accuracy: 0.7568
Epoch